# 9章

- 本章で用いる「[日本語Wikipedia入力誤りデータセット](https://nlp.ist.i.kyoto-u.ac.jp/?%E6%97%A5%E6%9C%AC%E8%AA%9EWikipedia%E5%85%A5%E5%8A%9B%E8%AA%A4%E3%82%8A%E3%83%87%E3%83%BC%E3%82%BF%E3%82%BB%E3%83%83%E3%83%88)」は現在バージョン2が公開されていますが、本章では[バージョン1](https://nlp.ist.i.kyoto-u.ac.jp/?%E6%97%A5%E6%9C%AC%E8%AA%9EWikipedia%E5%85%A5%E5%8A%9B%E8%AA%A4%E3%82%8A%E3%83%87%E3%83%BC%E3%82%BF%E3%82%BB%E3%83%83%E3%83%88v1)を用いています。

本章では、文章校正タスクのうち漢字の誤変換タスクを扱う。BERTでは、文章校正をトークンの分類問題として扱うことで実装できる。<br>
正しいトークンについては同じトークンを、間違っていると考えられるトークンにはBERTの語彙の中から正しいと予測したトークンを返す。

|誤変換|正しい文章|
|:-|:-|
|優勝|優勝|
|トロフィー|トロフィー|
|を|を|
|変換|返還|
|し|し|
|た|た|
|。|。|

一方で、以下のようにこの方法では扱えない文章が存在する。

|誤変換|正しい文章|
|:-|:-|
|投|当初|
|##書|は|
|は|、|
|、|実行|
|実行|を|

本章では、上記のような文章を取り扱わないこととする。

In [89]:
# 9-3

import os
import random
import unicodedata
import pandas as pd
from tqdm import tqdm
import pprint
import math

import torch
from torch.utils.data import DataLoader
from transformers import BertJapaneseTokenizer, BertForMaskedLM
import pytorch_lightning as pl

トークナイザを定義する。

In [90]:
# 9-4

class SC_tokenizer(BertJapaneseTokenizer):
    def encode_plus_tagged(
        self,
        wrong_text: str,
        correct_text: str,
        max_length: int = 128,
    ) -> dict:
        """
        ファインチューニング用
        誤変換を含む文章と正しい文章を渡し符号化し、
        誤変換文章のlabelsを正しい文章のinput_idsに置き換える
        """
        # 文章から直接符号化
        # tokenizer = BertJapaneseTokenizer.from_pretrained(model_name)で
        # インスタンス化したtokinzer(text)関数と同等
        wrong_encoding = self(
            wrong_text, max_length=max_length, padding='max_length', truncation=True
        )
        correct_encoding = self(
            correct_text, max_length=max_length, padding='max_length', truncation=True
        )
        # 正しい文章の符号をラベルとする
        wrong_encoding['labels'] = correct_encoding['input_ids']

        return wrong_encoding
    
    def encode_plus_untagged(
        self,
        text: str,
        max_length: int = None,
        return_tensors: str = 'pt',
    ) -> (dict, list):
        """
        文章をトークン化し、それぞれのトークンと文章中の文字列を対応付ける
        推論時にトークンごとのラベルを予測し、最終的に固有表現に変換する
        未知語や文章中の空白(MeCabにより消去される)に対しての処理が必要となる
        そのため、各トークンが元の文章のどの位置にあったかを特定しておく
        """
        words = self.word_tokenizer.tokenize(text) # MeCabで単語に分割
        tokens = []
        tokens_original = []
        # 単語をサブワードに分割してlistに格納
        for word in words:
            tokens_subword = self.subword_tokenizer.tokenize(word)
            tokens.extend(tokens_subword)
            # 未知語対応
            if tokens_subword[0] == '[UNK]':
                tokens_original.append(word)
            else:
                tokens_original.extend([token.replace("##", "") for token in tokens_subword])
        
        # トークンが文章中のどの位置にあるかを走査する
        position = 0
        spans = []
        for token in tokens_original:
            length = len(token)
            while True:
                if token != text[position: position + length]:
                    position += 1
                else:
                    spans.append([position, position + length])
                    position += length
                    break
        
        # トークンをID化する
        input_ids = self.convert_tokens_to_ids(tokens)
        # トークンIDを符号化する
        encoding = self.prepare_for_model(
            input_ids, 
            max_length=max_length,
            padding = 'max_length' if max_length else False,
            truncation = True if max_length else False,
        )
        sequence_length = len(encoding['input_ids']) # 符号化した文章の長さ
        # 先頭トークン[CLS]用のspanを追加する
        # このとき、次の[SEP]トークンを一緒に削除しておく
        spans = [[-1, -1]] + spans[:sequence_length - 2]
        # 末尾トークン[SEP]、末尾の空トークン[PAD]用のspanを追加する
        spans = spans + [[-1, -1]] * (sequence_length - len(spans))

        # 引数に応じてtorch.Tensor型に返還
        # 次元を追加する必要がある
        if return_tensors == 'pt':
            encoding = {key: torch.tensor([value]) for key, value in encoding.items()}
        
        return (encoding, spans)
    
    def convert_output_to_text(
        self, 
        text: str,
        labels_arg: list,
        spans_arg: list,
    ) -> str:
        """
        文章、各トークンのラベルの予測値、文章中での位置から
        予測された文章に変換する
        """
        # 文章の長さチェック
        assert len(labels_arg) == len(spans_arg)

        # 特殊トークンを取り除く
        labels = []
        spans = []
        for label, span in zip(labels_arg, spans_arg):
            if span[0] != -1:
                labels.append(label)
                spans.append(span)
        
        # モデルが予測した文章を生成する
        text_pred = ""
        position = 0
        for label, span in zip(labels, spans):
            start, end = span
            # 空白文字の処理
            if position != start:
                text_pred += text[position: start]
            token_pred = self.convert_ids_to_tokens(label) # labelをトークン化
            token_pred = token_pred.replace("##", "") # サブワードの##を削除
            token_pred = unicodedata.normalize('NFKC', token_pred) # 文字列の正規化
            text_pred += token_pred
            position = end
        return text_pred

定義したトークナイザの動きを確認する。

モデルとトークナイザの呼び出し

In [91]:
# 9-5

model_name = 'cl-tohoku/bert-base-japanese-whole-word-masking'
tokenizer = SC_tokenizer.from_pretrained(model_name)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BertJapaneseTokenizer'. 
The class this function is called from is 'SC_tokenizer'.


`encode_plus_tagged()`メソッドは、誤変換した文章と正しい文章が渡されたら、誤変換した文章を符号化し`labels`に正しい文章のトークンIDを付与する。

In [92]:
# 9-6

wrong_text = "優勝トロフィーを変換した"
correct_text = "優勝トロフィーを返還した"

encoding = tokenizer.encode_plus_tagged(
    wrong_text, correct_text, max_length=12,
)

In [93]:
pprint.pprint(encoding)

{'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0],
 'input_ids': [2, 759, 18204, 11, 4618, 15, 10, 3, 0, 0, 0, 0],
 'labels': [2, 759, 18204, 11, 8274, 15, 10, 3, 0, 0, 0, 0],
 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]}


`encode_plus_untagged()`メソッドは、文章を符号化し、空白や未知語を考慮した上でそれぞれのトークンの位置を返す。

In [94]:
# 9-7

wrong_text = "優勝トロフィーを変換した"
encoding, spans = tokenizer.encode_plus_untagged(
    wrong_text, return_tensors='pt'
)

In [95]:
encoding

{'input_ids': tensor([[    2,   759, 18204,    11,  4618,    15,    10,     3]]),
 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0]]),
 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1]])}

In [96]:
spans

[[-1, -1], [0, 2], [2, 7], [7, 8], [8, 10], [10, 11], [11, 12], [-1, -1]]

`convert_output_to_text()`関数は、文章とラベル列、各トークンの文章中の位置から予測した文章を出力する。

In [97]:
# 9-8

labels_pred = [2, 759, 18204, 11, 8274, 15, 10, 3]
text_pred = tokenizer.convert_output_to_text(
    wrong_text, labels_pred, spans
)

In [98]:
text_pred

'優勝トロフィーを返還した'

## 9.2 BERTにおける実装

`transformers.BertForMaskedLM`クラスを用いる。このクラスは各トークンに対してその市に入るトークンを語彙の中から選ぶもので、これは`transformers.BertForTokenClassification`でラベル数を語彙数としたものと入出力関係が同じと考えることができる。<br>
一方で、`transformers.BertForMaskedLM`は、事前学習でランダムに選ばれたトークンを`[MASK]`もしくはランダムなトークンに置き換え、またはそのままで、元のトークンが何であったかを予測するという学習が行われる。これは、文章校正タスクと類似しており、分類器の初期パラメータとして事前学習により得られたものを用いる`transformers.BertForMaskedLM`のほうが、`transformers.BertForTokenClassification`と比較してある程度妥当なパラメータを備えていることが期待されるためである。これにより、ファインチューニングの学習時間の短縮が期待できる。

In [99]:
# 9-9

model_name = 'cl-tohoku/bert-base-japanese-whole-word-masking'
model = BertForMaskedLM.from_pretrained(model_name)
model = model.cuda()

Some weights of the model checkpoint at cl-tohoku/bert-base-japanese-whole-word-masking were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


文章の符号化から予測された文章の出力までは、以下のように実装する。

In [100]:
# 9-10

text = "優勝トロフィーを変換した。"

# 符号化およびトークンの文章中の位置を取得
encoding, spans = tokenizer.encode_plus_untagged(
    text, return_tensors='pt'
)
encoding = {key: value.cuda() for key, value in encoding.items()}

# BERTに入力
with torch.no_grad():
    output = model(**encoding)
scores = output.logits
labels_pred = scores[0].argmax(axis=1).cpu().numpy().tolist()

text_pred = tokenizer.convert_output_to_text(
    text, labels_pred, spans
)

In [101]:
text_pred

'優勝トロフィーを獲得した。'

次に、誤変換した文章と正しい文章をBERTに入力して損失を計算する。入力時に、それぞれの文章を符号化し、正しい文章についてはラベル列を正とする。`tokenizer.encode_plus_tagged()`を用いることで、ラベル列を含んだデータを作成できる。

In [102]:
# 9-11

data = [
    {
        'wrong_text': '優勝トロフィーを変換した。',
        'correct_text': '優勝トロフィーを返還した。',
    },
    {
        'wrong_text': '人と森は強制している。',
        'correct_text': '人と森は共生している。',
    }
]

# データの符号化
max_length = 32
dataset_for_loader = []
for sample in data:
    wrong_text = sample['wrong_text']
    correct_text = sample['correct_text']
    encoding = tokenizer.encode_plus_tagged(
        wrong_text, correct_text, max_length=max_length
    )
    encoding = {key: torch.tensor(value) for key, value in encoding.items()}
    dataset_for_loader.append(encoding)

dataloader = DataLoader(dataset_for_loader, batch_size=2)

In [103]:
for batch in dataloader:
    encoding = {key: value.cuda() for key, value in batch.items()}
    output = model(**encoding)
    loss = output.loss

In [104]:
loss

tensor(13.2660, device='cuda:0', grad_fn=<NllLossBackward0>)

## 9.4 日本語Wikipedia誤りデータセット

本章では、京都大学の言語メディア研究室が作成した[日本語Wikipedia誤りデータセット]("https://nlp.ist.i.kyoto-u.ac.jp/?日本語Wikipedia入力誤りデータセット")を用いて学習を行う。このデータセットは、日本語Wikipediaの差分から間違った文章と正しい文章のペアを抽出している。

まずは、データセット(v2.0)をダウンロードする。

In [105]:
# 9-2

os.makedirs("data/chapter9", exist_ok=True)

In [110]:
# 9-12

!curl -o "data/chapter9/jwtd.tar.gz" "https://nlp.ist.i.kyoto-u.ac.jp/nl-resource/JWTD/jwtd_v2.0.tar.gz&name=JWTDv2.0.tar.gz"
!cd "data/chapter9" && tar -zxf "jwtd.tar.gz"

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   249  100   249    0     0   2480      0 --:--:-- --:--:-- --:--:--  2490
 17 94.0M   17 16.0M    0     0  1905k      0  0:00:50  0:00:08  0:00:42 1890k

100 94.0M  100 94.0M    0     0  1823k      0  0:00:52  0:00:52 --:--:-- 2291k


本データセットはJSON形式で、`pre_text`に修正前の文章、`post_text`に修正後の文章が格納されている。
さらに、`diffs`配列の中の`category`に入力誤りの種類(誤字、脱字など)が、`pre_str`が修正前、`post_str`が修正後の単語が格納されている。

本章では漢字誤変換のみを扱う。`category`が`kanji-conversion_a`となっているもののみを利用する。

まずデータを読み込む。

In [107]:
# 9-13

model_name = 'cl-tohoku/bert-base-japanese-whole-word-masking'
tokenizer = SC_tokenizer.from_pretrained(model_name)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BertJapaneseTokenizer'. 
The class this function is called from is 'SC_tokenizer'.


In [112]:
category_type = 'kanji-conversion_a'

def create_dataset(data_df: pd.DataFrame) -> dict:
    def _check_category(lst: list) -> bool:
        """
        data_df.diffsを走査して、
        - listの長さが2以下
        - すべてのcategoryがkanji-conversion_a
        のすべてを満たすときにTrue、そうでないときにFalseを返す
        """
        checked_lst = [dct['category'] == category_type for dct in lst]
        category_bool =  math.prod(checked_lst) # リスト内を掛け算してFalseがあれば0が返る
        length_bool = len(checked_lst) <= 2
        return bool(category_bool * length_bool)

    def _normalize(text: str) -> str:
        """
        文字列の正規化を行う
        """
        text = text.strip() # 改行文字や全角スペースを取り除く
        return unicodedata.normalize('NFKC', text) # NFKCで正規化

    # data_df.diffsのtypeをobject -> listに変換
    data_df.diffs = data_df.diffs.apply(lambda x: list(x))

    # data_dfからcategoryがkanji-conversion_aかつ誤変換が2以下の文章のみを抜き出す
    data_df = data_df[data_df['diffs'].apply(_check_category)].copy()

    # 文章の正規化
    data_df.pre_text = data_df.pre_text.apply(_normalize)
    data_df.post_text = data_df.post_text.apply(_normalize)

    return data_df[['pre_text', 'post_text']].to_dict(orient='records')

In [113]:
# データセットを読み込んで加工する
dir_path = "data/chapter9/jwtd_v2.0/"
train_df = pd.read_json(dir_path+"train.jsonl", orient='records', lines=True)
test_df = pd.read_json(dir_path+"test.jsonl", orient='records', lines=True)

# train/val
tmp_dataset = create_dataset(train_df)
random.shuffle(tmp_dataset)
train_size = int(len(tmp_dataset) * 0.8)
train_dataset = tmp_dataset[:train_size]
val_dataset = tmp_dataset[train_size:]
# test
test_dataset = create_dataset(test_df)

## 9.5 ファインチューニング

ファインチューニングのためのデータローダを定義する。

In [ ]:
# 9-14
def create_dataset_for_loader(tokenizer, dataset, max_length):
    """
    データセットをデータローダに入力可能な形式にする。
    """
    dataset_for_loader = []
    for sample in tqdm(dataset):
        wrong_text = sample['wrong_text']
        correct_text = sample['correct_text']
        encoding = tokenizer.encode_plus_tagged(
            wrong_text, correct_text, max_length=max_length
        )
        encoding = { k: torch.tensor(v) for k, v in encoding.items() }
        dataset_for_loader.append(encoding)
    return dataset_for_loader

tokenizer = SC_tokenizer.from_pretrained(MODEL_NAME)

# データセットの作成
max_length = 32
dataset_train_for_loader = create_dataset_for_loader(
    tokenizer, dataset_train, max_length
)
dataset_val_for_loader = create_dataset_for_loader(
    tokenizer, dataset_val, max_length
)

# データローダの作成
dataloader_train = DataLoader(
    dataset_train_for_loader, batch_size=32, shuffle=True
)
dataloader_val = DataLoader(dataset_val_for_loader, batch_size=256)

In [ ]:
# 9-15
class BertForMaskedLM_pl(pl.LightningModule):
        
    def __init__(self, model_name, lr):
        super().__init__()
        self.save_hyperparameters()
        self.bert_mlm = BertForMaskedLM.from_pretrained(model_name)
        
    def training_step(self, batch, batch_idx):
        output = self.bert_mlm(**batch)
        loss = output.loss
        self.log('train_loss', loss)
        return loss
        
    def validation_step(self, batch, batch_idx):
        output = self.bert_mlm(**batch)
        val_loss = output.loss
        self.log('val_loss', val_loss)
   
    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=self.hparams.lr)

checkpoint = pl.callbacks.ModelCheckpoint(
    monitor='val_loss',
    mode='min',
    save_top_k=1,
    save_weights_only=True,
    dirpath='model/'
)

trainer = pl.Trainer(
    gpus=1,
    max_epochs=5,
    callbacks=[checkpoint]
)

# ファインチューニング
model = BertForMaskedLM_pl(MODEL_NAME, lr=1e-5)
trainer.fit(model, dataloader_train, dataloader_val)
best_model_path = checkpoint.best_model_path

In [ ]:
# 9-16
def predict(text, tokenizer, bert_mlm):
    """
    文章を入力として受け、BERTが予測した文章を出力
    """
    # 符号化
    encoding, spans = tokenizer.encode_plus_untagged(
        text, return_tensors='pt'
    ) 
    encoding = { k: v.cuda() for k, v in encoding.items() }

    # ラベルの予測値の計算
    with torch.no_grad():
        output = bert_mlm(**encoding)
        scores = output.logits
        labels_predicted = scores[0].argmax(-1).cpu().numpy().tolist()

    # ラベル列を文章に変換
    predict_text = tokenizer.convert_bert_output_to_text(
        text, labels_predicted, spans
    )

    return predict_text

# いくつかの例に対してBERTによる文章校正を行ってみる。
text_list = [
    'ユーザーの試行に合わせた楽曲を配信する。',
    'メールに明日の会議の史料を添付した。',
    '乳酸菌で牛乳を発行するとヨーグルトができる。',
    '突然、子供が帰省を発した。'
]

# トークナイザ、ファインチューニング済みのモデルのロード
tokenizer = SC_tokenizer.from_pretrained(MODEL_NAME)
model = BertForMaskedLM_pl.load_from_checkpoint(best_model_path)
bert_mlm = model.bert_mlm.cuda()

for text in text_list:
    predict_text = predict(text, tokenizer, bert_mlm) # BERTによる予測
    print('---')
    print(f'入力：{text}')
    print(f'出力：{predict_text}')

In [ ]:
# 9-17
# BERTで予測を行い、正解数を数える。
correct_num = 0 
for sample in tqdm(dataset_test):
    wrong_text = sample['wrong_text']
    correct_text = sample['correct_text']
    predict_text = predict(wrong_text, tokenizer, bert_mlm) # BERT予測
   
    if correct_text == predict_text: # 正解の場合
        correct_num += 1

print(f'Accuracy: {correct_num/len(dataset_test):.2f}')

In [ ]:
# 9-18
correct_position_num = 0 # 正しく誤変換の漢字を特定できたデータの数
for sample in tqdm(dataset_test):
    wrong_text = sample['wrong_text']
    correct_text = sample['correct_text']
    
    # 符号化
    encoding = tokenizer(wrong_text)
    wrong_input_ids = encoding['input_ids'] # 誤変換の文の符合列
    encoding = {k: torch.tensor([v]).cuda() for k,v in encoding.items()}
    correct_encoding = tokenizer(correct_text)
    correct_input_ids = correct_encoding['input_ids'] # 正しい文の符合列
    
    # 文章を予測
    with torch.no_grad():
        output = bert_mlm(**encoding)
        scores = output.logits
        # 予測された文章のトークンのID
        predict_input_ids = scores[0].argmax(-1).cpu().numpy().tolist() 

    # 特殊トークンを取り除く
    wrong_input_ids = wrong_input_ids[1:-1]
    correct_input_ids =  correct_input_ids[1:-1]
    predict_input_ids =  predict_input_ids[1:-1]
    
    # 誤変換した漢字を特定できているかを判定
    # 符合列を比較する。
    detect_flag = True
    for wrong_token, correct_token, predict_token \
        in zip(wrong_input_ids, correct_input_ids, predict_input_ids):

        if wrong_token == correct_token: # 正しいトークン
            # 正しいトークンなのに誤って別のトークンに変換している場合
            if wrong_token != predict_token: 
                detect_flag = False
                break
        else: # 誤変換のトークン
            # 誤変換のトークンなのに、そのままにしている場合
            if wrong_token == predict_token: 
                detect_flag = False
                break

    if detect_flag: # 誤変換の漢字の位置を正しく特定できた場合
        correct_position_num += 1
        
print(f'Accuracy: {correct_position_num/len(dataset_test):.2f}')